In [1]:
from tqdm import tqdm
from pathlib import Path
from pysus import SIH
import pandas as pd
import sys
sys.path.append("..")

from pyopensus.storage.whandler_sus import HandlerSIH

PyTables is not installed. No support for HDF output.


In [21]:
def load_SIH_file(data_filename):
    """Load a SIH RD/CE parquet file, add metadata columns, and parse date fields.
    
    Parameters
    ----------
    data_filename : str or pathlib.Path
        Path to the parquet file to load.
    
    Returns
    -------
    pandas.DataFrame
        Loaded data with `FONTE`, `SEQUENCIA`, and parsed date columns.
    """
    
    prefix = Path(data_filename).stem[:2]
    uf = Path(data_filename).stem[2:4]
    mes_ano=Path(data_filename).stem
    mes_ano=mes_ano.replace(prefix,"").replace(uf,"")
   
    df = pd.read_parquet(data_filename, engine="fastparquet")
    nrows = df.shape[0]
    
    df['FONTE'] = [mes_ano for n in range(nrows)]
    
    # 'SEQUENCIA' tem valores repetidos
    df['SEQUENCIA'] = df['SEQUENCIA'].str.replace(" ","0")
    # df['FONTE_SEQUENCIA']= df['FONTE']+df['SEQUENCIA']
    
    # date columns...
    if prefix != "SP":
        for col in ["NASC", "DT_INTER", "DT_SAIDA"]:
            df[col] = pd.to_datetime(df[col], format="%Y%m%d", errors='coerce')
    return df

In [25]:
data_folder =  Path.home().joinpath("Workspace", "pyopensus", "data")
base_folder = Path.joinpath(data_folder,"sihsus")
warehouse_location = Path.joinpath(data_folder, "opendatasus")
warehouse_name = "SIHSUS_NORDESTE_NO_SERVICE.db"
warehouse_injector = HandlerSIH(warehouse_location, warehouse_name)

# warehouse_name = "SIHSUS_NORDESTE_NO_SERVICE.db"
parquet_location = base_folder

In [26]:
prefix_list = ["RD", "RJ", "SP"]
uf_list = ["CE"]
year_list = [2019]
month_list = [1]
sih = SIH().load()

files = sih.get_files(prefix_list, uf=uf_list, year=year_list, month=month_list);
filenames = [ parquet_location.joinpath(str(f).replace("dbc", "parquet")) for f in files]
for f in filenames:
    print(f, f.exists())

/home/humberto/Workspace/pyopensus/data/sihsus/RDCE1901.parquet True
/home/humberto/Workspace/pyopensus/data/sihsus/RJCE1901.parquet True
/home/humberto/Workspace/pyopensus/data/sihsus/SPCE1901.parquet True


In [27]:
for current_file in tqdm(filenames):
    fname = current_file.stem
    prefix = fname[:2]
    cur_df = load_SIH_file(current_file)
    warehouse_injector.insert_sih(cur_df, fname, prefix)
    

  0%|          | 0/3 [00:00<?, ?it/s]/opt/miniconda3/envs/opensus/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
 33%|███▎      | 1/3 [00:02<00:04,  2.32s/it]/opt/miniconda3/envs/opensus/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
 67%|██████▋   | 2/3 [00:02<00:01,  1.06s/it]/opt/miniconda3/envs/opensus/lib/python3.13/site-packages/numpy/_core/fromnumeric.py:57: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)
100%|██████████| 3/3 [00:14<00:00,  4.85s/it]


PosixPath('/home/humberto/Workspace/pyopensus/workflow/opendatasus/SIHSUS_NORDESTE_NO_SERVICE.db')

In [29]:
pwd.joinpath("opendatasus","SIHSUS_NORDESTE_NO_SERVICE.db")

NameError: name 'pwd' is not defined

In [30]:
Path(pwd)

NameError: name 'pwd' is not defined